<a href="https://colab.research.google.com/github/HebatallaTarek/Empathy-Mental-Health/blob/master/Pre_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**bold text**
# Configurations







In [ ]:
import os
import pandas as pd
import tqdm
import math

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#Set the path to the data folder, datafile and output folder and files
root_folder = '/content/drive/My Drive'
data_folder = os.path.abspath(os.path.join(root_folder, 'Empathy Detection Project/empathy_dataset'))
model_folder = os.path.abspath(os.path.join(root_folder, 'Empathy Detection Project/pre-training model'))
output_folder = os.path.abspath(os.path.join(root_folder, 'Empathy Detection Project/pre-tarining output'))
tokenizer_folder = os.path.abspath(os.path.join(root_folder, 'Empathy Detection Project/tokenizer'))

test_filename='emotional-reactions_Test-reddit.csv'
datafile= 'emotional-reactions-reddit.csv'
outputfile = 'emotionalreactionafterMLM.csv'

datafile_path = os.path.abspath(os.path.join(data_folder,datafile))
testfile_path = os.path.abspath(os.path.join(data_folder,test_filename))
outputfile_path = os.path.abspath(os.path.join(output_folder,outputfile))

# Load Training and Testing Datasets

In [ ]:
# Load the train dataset
train_df=pd.read_csv(datafile_path, header=0, usecols=[2,3])
# Show the count of rows
print('Num Examples: ',len(train_df))
print('Null Values\n', train_df.isna().sum())
print(train_df)
# Drop rows with Null values 
train_df.dropna(inplace=True)
print('Num Examples: ',len(train_df))

Num Examples:  3084
Null Values
 seeker_post      0
response_post    0
dtype: int64
                                            seeker_post                                      response_post
0         Help. Help me. I dunno what I'm doing anymore  That's pretty vague, do you not know what you'...
1     I'm done saying I love you to her because I do...  idk what a Red pill means exactly but my advic...
2     Always feel like I'm being criticized and mock...  I think it's social anxiety , that creates par...
3     My diet becomes fucked when i get depressed.. ...  By any chance do you think you're in a loop. J...
4     I hate not knowing why. I was diagnosed with d...  depression. not sadness which is caused by som...
...                                                 ...                                                ...
3079  does anyone else keep forgetting stuff the nee...  All day, every day. It's definitely not just y...
3080  What does depression feel like?. Honest questi...  lik

In [ ]:
# Load the test dataset 
test_df=pd.read_csv(testfile_path, header=0)
print('Num Examples: ',len(test_df))
print('Null Values\n', test_df.isna().sum())
print(test_df)
# there are no null values

Num Examples:  1001
Null Values
 seeker_post      0
response_post    0
dtype: int64
                                            seeker_post                                      response_post
0     Suicide. I think about suicide every day. Been...  Do you know the author Tom Robbins? He recentl...
1     I hate when people say that when I kill myself...  Thanks for sharing that. I agree that telling ...
2     Does anyone notice a change in libido during e...  Now this is an odd one. I felt a drop in my se...
3     I don’t have the ability to cry anymore. No ma...  I experienced smth similar during the last two...
4     Do happy occasions make anyone else feel more ...  I always see couples and hate them. Not becaus...
...                                                 ...                                                ...
996   does anyone else keep forgetting stuff the nee...  All day, every day. It's definitely not just y...
997   What does depression feel like?. Honest questi...  lik

# Create the dataset to train a tokenizer

In [ ]:


# Drop the files from the output dir
txt_files_dir = "./text_split"
!rm -rf {txt_files_dir}
!mkdir {txt_files_dir}



In [ ]:
# Store values in a dataframe column (Series object) to files, one file per record
def column_to_files(column, prefix, txt_files_dir):
    # The prefix is a unique ID to avoid to overwrite a text file
    i=prefix
    #For every value in the df, with just one column
    for row in column.to_list():
      # Create the filename using the prefix ID
      file_name = os.path.join(txt_files_dir, str(i)+'.txt')
      try:
        # Create the file and write the column text to it
        f = open(file_name, 'wb')
        f.write(row.encode('utf-8'))
        f.close()
      except Exception as e:  #catch exceptions(for eg. empty rows)
        print(row, e) 
      i+=1
    # Return the last ID
    return i

In [ ]:
data = train_df["response_post"]
# Removing the end of line character \n
data = data.replace("\n"," ")
# Set the ID to 0
prefix=0
# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)
# Print the last ID
print(prefix)

3084


In [ ]:
data = test_df["response_post"]
# Removing the end of line character \n
data = data.replace("\n"," ")
print(len(data))
# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)
print(prefix)

1001
4085


In [ ]:
data = train_df["seeker_post"]
data = data.replace("\n"," ")
print(len(data))
prefix = column_to_files(data, prefix, txt_files_dir)
print(prefix)

3084
7169


# Train Tokenizer

In [ ]:
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1
import tensorflow as tf 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-q6r9dpi5
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-q6r9dpi5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
tokenizers                    0.10.3
transformers                  4.13.0.dev0


In [ ]:
paths = [str(x) for x in Path(".").glob("text_split/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer.train(files=paths, vocab_size=8192, min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])

In [ ]:
tokenizer

Tokenizer(vocabulary_size=8192, model=ByteLevelBPE, add_prefix_space=False, lowercase=True, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [ ]:
# Tokenizer. save_model () is outdated so we used .save_pretrained()
tokenizer.save_pretrained(tokenizer_folder)

('/content/drive/My Drive/Empathy Detection Project/tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/Empathy Detection Project/tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/Empathy Detection Project/tokenizer/vocab.json',
 '/content/drive/My Drive/Empathy Detection Project/tokenizer/merges.txt',
 '/content/drive/My Drive/Empathy Detection Project/tokenizer/added_tokens.json',
 '/content/drive/My Drive/Empathy Detection Project/tokenizer/tokenizer.json')

In [ ]:
# Create the tokenizer using vocab.json and mrege.txt files
tokenizer = ByteLevelBPETokenizer(
    os.path.abspath(os.path.join(tokenizer_folder,'vocab.json')),
    os.path.abspath(os.path.join(tokenizer_folder,'merges.txt'))
)

In [ ]:
# Prepare the tokenizer
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("Do you know the author Tom Robbins? He recently wrote a memoir. In it, he said Everything can change completely at any moment. This is still the best argument I know against suicide. You still have plenty of life to reap, I guarantee it. What you're going through is temporary, even if it's been a long time, even if it seems like there's no end in sight. You're going to be ok.")

Encoding(num_tokens=116, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
tokenizer.encode("Do you know the author Tom Robbins? He recently wrote a memoir. In it, he said Everything can change completely at any moment. This is still the best argument I know against suicide. You still have plenty of life to reap, I guarantee it. What you're going through is temporary, even if it's been a long time, even if it seems like there's no end in sight. You're going to be ok.").tokens

['<s>',
 'D',
 'o',
 'Ġyou',
 'Ġknow',
 'Ġthe',
 'Ġauthor',
 'Ġ',
 'T',
 'om',
 'Ġ',
 'R',
 'o',
 'bb',
 'ins',
 '?',
 'Ġ',
 'H',
 'e',
 'Ġrecently',
 'Ġwrote',
 'Ġa',
 'Ġme',
 'mo',
 'ir',
 '.',
 'Ġ',
 'I',
 'n',
 'Ġit',
 ',',
 'Ġhe',
 'Ġsaid',
 'Ġ',
 'E',
 'very',
 'thing',
 'Ġcan',
 'Ġchange',
 'Ġcompletely',
 'Ġat',
 'Ġany',
 'Ġmoment',
 '.',
 'Ġ',
 'T',
 'h',
 'is',
 'Ġis',
 'Ġstill',
 'Ġthe',
 'Ġbest',
 'Ġargument',
 'Ġ',
 'I',
 'Ġknow',
 'Ġagainst',
 'Ġsuicide',
 '.',
 'Ġ',
 'Y',
 'ou',
 'Ġstill',
 'Ġhave',
 'Ġplenty',
 'Ġof',
 'Ġlife',
 'Ġto',
 'Ġreap',
 ',',
 'Ġ',
 'I',
 'Ġguarantee',
 'Ġit',
 '.',
 'Ġ',
 'W',
 'hat',
 'Ġyou',
 "'re",
 'Ġgoing',
 'Ġthrough',
 'Ġis',
 'Ġtemporary',
 ',',
 'Ġeven',
 'Ġif',
 'Ġit',
 "'s",
 'Ġbeen',
 'Ġa',
 'Ġlong',
 'Ġtime',
 ',',
 'Ġeven',
 'Ġif',
 'Ġit',
 'Ġseems',
 'Ġlike',
 'Ġthere',
 "'s",
 'Ġno',
 'Ġend',
 'Ġin',
 'Ġsight',
 '.',
 'Ġ',
 'Y',
 'ou',
 "'re",
 'Ġgoing',
 'Ġto',
 'Ġbe',
 'Ġok',
 '.',
 '</s>']

# Train Language Model


In [ ]:
TRAIN_BATCH_SIZE = 8   # input batch size for training (default: 64)
VALID_BATCH_SIZE = 8    # input batch size for testing (default: 1000)
TRAIN_EPOCHS = 13       # number of epochs to train (default: 10)
LEARNING_RATE = 1e-4    # learning rate (default: 0.001)
WEIGHT_DECAY = 0.01
SEED = 12               # random seed (default: 42)
MAX_LEN = 128
SUMMARY_LEN = 7

In [ ]:
!nvidia-smi

Fri Nov 12 02:53:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    74W / 149W |    144MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
gpu_available = tf.test.is_gpu_available()
print(gpu_available)

True


In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=8192,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

Num parameters:  49816064


In [ ]:
from transformers import RobertaTokenizerFast
# Create the tokenizer from a trained one
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=MAX_LEN)

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='/content/drive/My Drive/Empathy Detection Project/tokenizer', vocab_size=8192, model_max_len=128, is_fast=True, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [ ]:
from torch.utils.data.dataset import Dataset

numeric_feature_names = ['seeker_post', 'response_post']
numeric_features = train_df[numeric_feature_names]
dataset = tf.data.experimental.make_csv_dataset(
  datafile_path,
  batch_size=8,
  select_columns = ["seeker_post","response_post"])
#Dataset = tf.data.experimental.make_csv_dataset(datafile_path, batch_size=8)


#Dataset = tf.data.Dataset.from_tensor_slices(train_df["seeker_post"],train_df["response_post"])

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer):
        # or use the RobertaTokenizer from `transformers` directly.

        self.examples = []
        
        for example in df.values:
            x=tokenizer.encode_plus(example, max_length = MAX_LEN, truncation=True, padding=True)
            self.examples += [x.input_ids]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])



In [ ]:
# Create the train and evaluation dataset
train_dataset = CustomDataset(train_df['response_post'], tokenizer)
eval_dataset = CustomDataset(test_df['response_post'], tokenizer)

In [ ]:
from transformers import DataCollatorForLanguageModeling

# Define the Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


In [ ]:
from transformers import Trainer, TrainingArguments

print(model_folder)
# Define the training arguments
training_args = TrainingArguments(
    output_dir=model_folder,
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    save_steps=8192,
    #eval_steps=4096,
    save_total_limit=1,
)
# Create the trainer for our model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    #prediction_loss_only=True,
)

/content/drive/My Drive/Empathy Detection Project/pre-training model


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 3084
  Num Epochs = 13
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5018


Epoch,Training Loss,Validation Loss
1,No log,6.248062
2,6.430000,5.998797
3,6.094200,5.811765
4,5.814100,5.747410
5,5.814100,5.656811
6,5.720300,5.599064
7,5.658800,5.623189
8,5.605300,5.599012
9,5.605300,5.416981
10,5.537500,5.419897


***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5018, training_loss=5.703292490904813, metrics={'train_runtime': 1237.2071, 'train_samples_per_second': 32.405, 'train_steps_per_second': 4.056, 'total_flos': 1195687373488128.0, 'train_loss': 5.703292490904813, 'epoch': 13.0})

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8


Perplexity: 198.78


In [ ]:
trainer.save_model(model_folder)

Saving model checkpoint to /content/drive/My Drive/Empathy Detection Project/pre-training model
Configuration saved in /content/drive/My Drive/Empathy Detection Project/pre-training model/config.json
Model weights saved in /content/drive/My Drive/Empathy Detection Project/pre-training model/pytorch_model.bin


In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model_folder,
    tokenizer=tokenizer_folder
)


loading configuration file /content/drive/My Drive/Empathy Detection Project/pre-training model/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/drive/My Drive/Empathy Detection Project/pre-training model",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 8192
}

loading configuration file /content/drive/My Drive/Empathy Detection Project/pre-training model/config.json
Model config RobertaConfi

In [ ]:
fill_mask("Yeah, this <mask> is a strong one. Felt it for a lot of my childhood and early adulthood. The truth is that I never dealt with it constructively, or at least in a way that could help me get to the root of that feeling. Part of the reason I think you and I are on this in the first place is because we were never taught how to deal with that feeling constructively. In my case, I used constant denial and self-numbing ... At some point, I just assumed the pit was a part of me I could choose to ignore. You seem to be actively trying to experience it. You are engaging your sadness, which is objectively good for working through depression. It may not be what you might call 'conventional' but everyone does their thing a little differently :) If what you are doing no longer works for you though, it might be best to start looking for other (healthy) ways of working through that discomfort. Personally, when I feel that pit, I know what thoughts tend to follow it. Fortunately, my therapists (and my stay at hospital) helped me come up with ways to attack that pit when it hit me hard. In order to redirect those thoughts that create it, I try to reach out to anyone in my support net. It's hard because I feel like they won't give a shit, but most of the time, they do (but I often choose to ignore that reality). Or I will simply write some positive words here, on this. Reaching out to others here in positive ways is not just for others, but for my own sake as well. As you put so well, giving that pit a 'purpose'. But don't cover that feeling up. Continue to engage with it, as best as you can. It's there for a reason. Hopefully, the more you work with it, the easier it is to process and let go.")

[{'score': 0.08077195286750793,
  'sequence': "Yeah, this, is a strong one. Felt it for a lot of my childhood and early adulthood. The truth is that I never dealt with it constructively, or at least in a way that could help me get to the root of that feeling. Part of the reason I think you and I are on this in the first place is because we were never taught how to deal with that feeling constructively. In my case, I used constant denial and self-numbing... At some point, I just assumed the pit was a part of me I could choose to ignore. You seem to be actively trying to experience it. You are engaging your sadness, which is objectively good for working through depression. It may not be what you might call 'conventional' but everyone does their thing a little differently :) If what you are doing no longer works for you though, it might be best to start looking for other (healthy) ways of working through that discomfort. Personally, when I feel that pit, I know what thoughts tend to follo